In [85]:
import pandas as pd
import seaborn as sns
import numpy as np

In [86]:
df = pd.read_csv('./all.csv')
df = df.dropna()

# EDA

In [87]:
print(df.columns)
print(len(df))

Index(['docket_no', 'offenses', 'offense_date', 'arrest_dt', 'case_status',
       'arresting_officer', 'attorney', 'dob', 'zip', 'bail_set_by',
       'bail_amount', 'bail_paid', 'bail_date', 'bail_type',
       'prelim_hearing_dt', 'prelim_hearing_time'],
      dtype='object')
11565


In [88]:
df.iloc[4].offenses

"['Manufacture, Delivery, or Possession With Intent to Manufacture or Deliver', 'Manufacture, Delivery, or Possession With Intent to Manufacture or Deliver', 'Conspiracy', 'Conspiracy', 'Int Poss Contr Subst By Per Not Reg', 'Int Poss Contr Subst By Per Not Reg', 'Poss Of Marijuana', 'Poss Of Marijuana']"

### Parse offenses

looks like list format but this is actually a string

In [89]:
df.offenses.head()

0    ['Rape Forcible Compulsion', 'Rape Forcible Co...
1    ['Aggravated Assault - Attempts to cause SBI o...
2    ['Simple Assault', 'Simple Assault', 'Reckless...
3    ['Contraband/Controlled Substance', 'Contraban...
4    ['Manufacture, Delivery, or Possession With In...
Name: offenses, dtype: object

In [90]:
b = df["offenses"].str[1:-1].str.replace(",","").str.split("'", expand=True)
pd.set_option('display.max_columns', None)
b.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216
0,,Rape Forcible Compulsion,,Rape Forcible Compulsion,,Sexual Assault,,Sexual Assault,,Indec Asslt-W/O Cons Of Other,,Indec Asslt-W/O Cons Of Other,,Simple Assault,,Simple Assault,,False Imprisonment,,False Imprisonment,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,,Aggravated Assault - Attempts to cause SBI or ...,,Aggravated Assault - Attempts to cause SBI or ...,,Strangulation - Applying Pressure to Throat or...,,Strangulation - Applying Pressure to Throat or...,,Poss Instrument Of Crime W/Int,,Poss Instrument Of Crime W/Int,,Terroristic Threats W/ Int To Terrorize Another,,Terroristic Threats W/ Int To Terrorize Another,,Simple Assault,,Simple Assault,,Recklessly Endangering Another Person,,Recklessly Endangering Another Person,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,,Simple Assault,,Simple Assault,,Recklessly Endangering Another Person,,Recklessly Endangering Another Person,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Non

problem here, duplicate columns

In [91]:
b['combined'] = b.values.tolist()
parsed = []

for entry in b['combined']:
    #drop duplicates and also check for null values
    c = np.unique([x for x in entry if ( (x != None) & (x not in [' ', '']))]).tolist()
    parsed.append(c)

df['parsed_offenses'] = parsed

In [92]:
df["bail_paid"] = df["bail_paid"].astype(int)
df["bail_amount"] = df["bail_amount"].astype(int)

In [93]:
all_crimes = []
for i in parsed:
    for j in i:
        all_crimes.append(j)
s_all = pd.Series(all_crimes)

In [94]:
pd.option_context('display.max_rows', None, 'display.max_columns', None)
s_all.value_counts().head(200)

Simple Assault                                                                                3396
Int Poss Contr Subst By Per Not Reg                                                           3196
Recklessly Endangering Another Person                                                         3179
Poss Instrument Of Crime W/Int                                                                2129
Conspiracy                                                                                    2103
Manufacture Delivery or Possession With Intent to Manufacture or Deliver                      2028
Receiving Stolen Property                                                                     1781
Theft By Unlaw Taking-Movable Prop                                                            1576
Firearms Not To Be Carried W/O License                                                        1294
Carry Firearms Public In Phila                                                                1272
Terroristi

In [95]:
# df.to_csv("parsed1.csv")

### grouping by zip code

In [96]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)
df['zip'].value_counts().head(100)

#take only first part of zipcode
df['zip'] = df['zip'].str.split("-", expand=True)[0]

df['zip'] = df['zip'].apply(lambda x: '0'+str(x) if len(str(x))==4 else x)

In [97]:
len(df)

11565

In [112]:
df_zip_mean = df.groupby('zip').mean().rename(columns = {"bail_amount":"mean_bail_amount", "bail_paid":"mean_bail_paid"})
df_zip_med = df.groupby('zip').median().rename(columns = {"bail_amount":"med_bail_amount", "bail_paid":"med_bail_paid"})

df_zip = pd.concat([df_zip_mean, df_zip_med], axis = 1)

In [113]:
df_zip.head()

,mean_bail_amount,mean_bail_paid,med_bail_amount,med_bail_paid
zip,,,,
01742,0.0,0.0,0,0
01920,0.0,0.0,0,0
02324,250000.0,0.0,250000,0
02893,250000.0,0.0,250000,0
06415,0.0,0.0,0,0
